#### Análise de Sentimentos
    Resumo:
    Link da documentação:
    Link do repositório:
    
Integrantes: <br>
    Bruno Sampaio <br>
    Eric Saboia <br>
    Gabriel Puente <br>
    Israel Junior <br>

#### 1. Importações para o projeto

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import csv

from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter

#### 2. Importações dos dados e Análise Exploratória

In [ ]:
tweets = pd.read_excel('dados.xlsx', sheet_name='DS-PN-ID', encoding='utf-8')
tweets.shape

In [ ]:
tweets.head()

In [ ]:
tweets.describe()

#### 3. Técnicas Genéricas de Tratamento de dados
    Funções simples para tratar dados

In [ ]:
def transformacao_minuscula(texto):
    texto = texto.str.lower()
    return texto

In [ ]:
def separa_atributos_rotulos(dados):
    atributos = tweets.Text
    rotulos = tweets.Sentimento
    return atributos, rotulos

#### 4. Técnicas de Processamento de Linguagem Natural (PLN)
    Funções de aplicações das técnicas de PLN

In [ ]:
def vetorizar_texto(texto, tradutor, stemmer):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1
    return vetor

In [ ]:
def pre_processamento(dados):
    textosTokenizados = [nltk.tokenize.word_tokenize(frase) for frase in dados]

    stopwords = nltk.corpus.stopwords.words('portuguese')
    
    stemmer = nltk.stem.RSLPStemmer();       
    
    dicionario = set()
    for lista in textosTokenizados:
        validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra) > 2]
        dicionario.update(validas)
    

    totalDePalavras = len(dicionario)
    
    tuplas = zip(dicionario, range(totalDePalavras))
    tradutor = {palavra: indice for palavra, indice in tuplas}

    vetoresDeTexto = [vetorizar_texto(texto, tradutor, stemmer) for texto in textosTokenizados]
    #print(tradutor)
    return vetoresDeTexto, dicionario

In [ ]:
def pre_processamento_validacao(dados, dicionario):
    #aplica a tokenização
    textosTokenizados = [nltk.tokenize.word_tokenize(frase) for frase in dados]

    #aplica as stop words, isto é, remove os pronomes e etc.
    stopwords = nltk.corpus.stopwords.words('portuguese')
    
    #aplica steem, isto é, deixa a palavra na forma raiz
    stemmer = nltk.stem.RSLPStemmer();
    
    totalDePalavras = len(dicionario)
    #print(totalDePalavras)
    #print(dicionario)
    tuplas = zip(dicionario, range(totalDePalavras))
    tradutor = {palavra: indice for palavra, indice in tuplas}

    vetoresDeTexto = [vetorizar_texto(texto, tradutor, stemmer) for texto in textosTokenizados]

    return vetoresDeTexto

#### 5.  Válidações e métodos utilizados

##### 5.1.  Holdout
    Divisao dos dados: 70 treino /30 teste

In [ ]:
def holdoutValidacao(atributos, rotulos):
    return train_test_split(atributos, rotulos, test_size=0.3, random_state=42)

##### 5.2.  Cross-Validation K-Fold
    K = 5 (será utilizada técnicas de otimizações para encontrar o melhor K)

##### 5.2.  Cross-Validation K-Fold (Estrátificado)
    K = 5 (será utilizada técnicas de otimizações para encontrar o melhor K)

#### 6.  Modelos e algoritmos

#### 7.  Processamento
    Os dados coletados estão na váriavel 'tweets'

In [ ]:
atributo, rotulos = separa_atributos_rotulos(tweets)
atributo = transformacao_minuscula(atributos)
treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes = holdoutValidacao(atributo, rotulos)
treino_dados, dicionario = pre_processamento(treino_dados)

